# Run Yolo V5

## Clone Git, Install Requirements

In [ ]:
!git clone https://github.com/chia56028/Street-View-House-Numbers-Detection  # clone repo
%cd Street-View-House-Numbers-Detection
%pip install -r requirements.txt  # install dependencies

%pip install torch==1.7.0+cu110 torchvision==0.8.1+cu110 torchaudio===0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

## Download SVHN Dataset, Prepare Data
This is only necessary if you are training the model from scratch

In [ ]:
%cd //content/Street-View-House-Numbers-Detection/data/svhn

torch.hub.download_url_to_file('http://ufldl.stanford.edu/housenumbers/train.tar.gz', 'train.tar.gz')
torch.hub.download_url_to_file('http://ufldl.stanford.edu/housenumbers/test.tar.gz', 'test.tar.gz')

!tar -xvzf train.tar.gz -C .
!tar -xvzf test.tar.gz -C .

%cp -R test valid

In [ ]:
%cd //content/Street-View-House-Numbers-Detection/data/svhn

!git clone https://github.com/skj10/utility-meter  # clone repo

%cp utility-meter/mat_to_yolo_arg_parse.py ./mat_to_yolo_arg_parse.py
%cp utility-meter/trim_data.py ./trim_data.py

!python trim_data.py --number 5000 --folder valid

!python mat_to_yolo_arg_parse.py --number 33402 --folder train
!python mat_to_yolo_arg_parse.py --number 13068 --folder test
!python mat_to_yolo_arg_parse.py --number 5000 --folder valid

## Train Model or Unzip Weights
**Training a model from scratch:** Mark condition in if...else statement as True. 

**Already moved runs.zip file into main directory:** Mark
if...else condition as False.

In [ ]:
%cd //content/Street-View-House-Numbers-Detection
if (False):
  !python train.py --img 320 --batch 16 --epochs 50 --data svhn.yaml --weights yolov5x.pt
  !zip -r runs.zip runs
else:
  !unzip runs.zip

## Unzip Utility Meter Images, Run Inference

In [ ]:
%cd //content/Street-View-House-Numbers-Detection/data/svhn
!unzip testMeters.zip

In [ ]:
%cd //content/Street-View-House-Numbers-Detection
!python detect.py --source data/svhn/testMeters/ --weights runs/train/exp/weights/best.pt --conf 0.50 --save-txt --save-conf

## Visualize

In [ ]:
%cd //content/Street-View-House-Numbers-Detection

Image(filename='runs/train/exp/train_batch0.jpg', width=800)  # train batch 0 mosaics and labels
Image(filename='runs/train/exp/test_batch0_labels.jpg', width=800)  # test batch 0 labels
Image(filename='runs/train/exp/test_batch0_pred.jpg', width=800)  # test batch 0 predictions

from utils.plots import plot_results 
plot_results(save_dir='runs/train/exp')  # plot all results*.txt as results.png
Image(filename='runs/train/exp/results.png', width=800)